## Logistic Regression: 

In this notebook, I'll be implementing a Logistic regression model on the Breast Cancer Dataset from Sklearn. The reason I'm using already prepared datasets is because the main focus is to develop my skills using PyTorch not necessarily feature reduction/EDA. 

So whats different here? Surely Logistic Regression has a very similar implementation to Linear regression? Indeed it does, however the implementation shows off an important part of PyTorch, in particular inheritance. 

PyTorch is a deep learning library and takes advantage of inheritance to allow us to build customized, complicated Neural Nets. I'll demonstrate this inheritance behaviour through Logistic Regression. 

Import Modules:

In [85]:
import pandas as pd
import torch
from torch import nn as nn
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split

Obtain Data from Sklearn:

In [86]:
X_df, y_df = datasets.load_breast_cancer(as_frame=True, return_X_y=True)
X_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


From the data we can see features have different scales, hence we should normalise the design matrix. 

Split datasets into training and testing:

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=1234)
normalizer = StandardScaler()
X_train = normalizer.fit_transform(X_train)
X_test = normalizer.transform(X_test)

Cast numpy arrays into tensors. Recall "double/ float64" precision tends to cause issues for PyTorch.

In [92]:
X_train = torch.from_numpy(X_train.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32).values)
X_test = torch.from_numpy(X_test.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32).values)
print(X_train.shape, y_train.shape)

torch.Size([455, 30]) torch.Size([455])


Reshaping tensors:

In [93]:
number_of_features = X_train.shape[1]
y_train = torch.reshape(y_train, shape=(-1,1))
y_test = torch.reshape(y_test,shape=(-1,1))

Normalize Data to improve G.D speed, I normalise the training set and exclude the test set from impacting the means/sds, preventing data leakage in our model:

In [94]:
X_train = nn.functional.normalize(X_train)
X_test = nn.functional.normalize(X_test)

The following shows how we can build a custom model within PyTorch. In this case we'll obviously be creating a Logistic Regression model.

In [95]:
class LogisticRegression(nn.Module):
    def __init__(self, input_features):
        super().__init__()
        # We setup a layer to feedforward. 
        self.linear_layer = nn.Linear(input_features, 1)
    
    def forward(self, x):
        return torch.sigmoid(self.linear_layer(x))

Now that we've created an instance of our model, we can apply Gradient Descent. 

In [96]:
model = LogisticRegression(number_of_features)
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_func = nn.BCELoss()
num_iters = 500

In [97]:
for epoch in range(1, num_iters+1):
    predictions = model(X_train)
    loss = loss_func(predictions, y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if epoch % 100 == 0:
        print(f' The Loss for Epoch: {epoch} is: {loss.item():.3f}')


 The Loss for Epoch: 100 is: 0.579
 The Loss for Epoch: 200 is: 0.523
 The Loss for Epoch: 300 is: 0.478
 The Loss for Epoch: 400 is: 0.441
 The Loss for Epoch: 500 is: 0.410


Lets review the model accuracy, ie the recall on the test sets: 

In [98]:
with torch.no_grad():
    test_predict = model(X_test)
    # Classify based on probabilities of belonging in each class.
    test_predict_binary = test_predict.round()
    errors = torch.abs(test_predict_binary - y_test)
    print(torch.sum(test_predict_binary.eq(y_test))/y_test.shape[0])
    print(f'The model Accuracy/Recall is: {(1-torch.sum(errors)/errors.shape[0]) * 100 :.2f}%')

tensor(0.8947)
The model Accuracy/Recall is: 89.47%
